# 🚀 API de Procesamiento de Imágenes con OCR

Este notebook te permite ejecutar tu API de manera interactiva usando Jupyter.

## 📋 Características:
- Procesamiento de imágenes y PDFs
- Extracción de texto con Tesseract
- Detección de layout con LayoutParser
- API REST con FastAPI
- Interfaz interactiva


## 🔧 Configuración Inicial


In [ ]:
# Importar librerías necesarias
import os
import sys
from pathlib import Path
import uvicorn
import threading
import time
import requests
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from IPython.display import FileLink

# Agregar el directorio actual al path
sys.path.append(str(Path.cwd()))

print("✅ Librerías importadas correctamente")


In [ ]:
# Configurar variables de entorno
os.environ['HOST'] = '0.0.0.0'
os.environ['PORT'] = '8080'  # Puerto diferente para evitar conflictos
os.environ['DEBUG'] = 'True'
os.environ['UPLOAD_DIR'] = 'temp_uploads'
os.environ['MAX_FILE_SIZE'] = '10485760'
os.environ['TESSERACT_PATH'] = '/usr/bin/tesseract'
os.environ['FAST_MODE'] = 'True'

print("✅ Variables de entorno configuradas")


In [ ]:
# Importar la aplicación FastAPI
from main import app
from config import settings

print(f"✅ API cargada correctamente")
print(f"   Host: {settings.HOST}")
print(f"   Puerto: {settings.PORT}")
print(f"   Debug: {settings.DEBUG}")


## 🚀 Iniciar el Servidor


In [ ]:
# Función para iniciar el servidor en un hilo separado
def start_server():
    """Iniciar el servidor FastAPI en un hilo separado"""
    try:
        uvicorn.run(
            app,
            host=settings.HOST,
            port=settings.PORT,
            log_level="info",
            access_log=True
        )
    except Exception as e:
        print(f"❌ Error iniciando servidor: {e}")

# Variable global para controlar el servidor
server_thread = None
server_running = False

def start_api_server():
    """Iniciar la API"""
    global server_thread, server_running
    
    if server_running:
        print("⚠️ El servidor ya está ejecutándose")
        return
    
    print("🚀 Iniciando servidor API...")
    server_thread = threading.Thread(target=start_server, daemon=True)
    server_thread.start()
    server_running = True
    
    # Esperar un poco para que el servidor se inicie
    time.sleep(3)
    
    # Verificar que el servidor esté funcionando
    try:
        response = requests.get(f"http://localhost:{settings.PORT}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Servidor iniciado correctamente")
            print(f"🌐 API disponible en: http://localhost:{settings.PORT}")
            print(f"📚 Documentación: http://localhost:{settings.PORT}/docs")
            print(f"🔍 Health Check: http://localhost:{settings.PORT}/health")
        else:
            print("⚠️ Servidor iniciado pero no responde correctamente")
    except Exception as e:
        print(f"⚠️ No se pudo verificar el servidor: {e}")
        print("💡 El servidor puede estar iniciándose, intenta de nuevo en unos segundos")

# Botón para iniciar el servidor
start_button = widgets.Button(
    description="🚀 Iniciar API",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

start_button.on_click(lambda b: start_api_server())
display(start_button)
